In [9]:

#Importar librerias? Pues claro que me importan!
import numpy as np
from numpy import random
from random import choices 
import os 


In [10]:

def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.6, 0.4]
    n = choices(estado, peso_de_decision)
    return n[0]

def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [0,1]
        peso = [0.87, 0.13] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad <= 65):
        valoracion = [0,1]
        peso = [0.99, 0.01] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso) #n es un vector cuyo primer elemento es la valoracion 
    return n[0]
    

In [18]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugaroriginal, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugaroriginal
            self.lugaroriginal=lugaroriginal
            self.diasestado=0 #esto se usa?
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            self.edificiomuerte=-1
            
             
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni) + "," +str(self.edad)            

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def esta_sano(self):
            return (self.estado == "sano") or (self.estado == "inmune")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible()):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].contagiarse(posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual)
                           # print(ciudad[self.lugar][i].cambioEstado)
                            self.contadorInfecciones +=1
                            
                            
        def contagiarse(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual):
            self.estado=self.estadosposibles[1]
            self.diasestado=0
            self.cambioEstado=diaactual+(random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2)
                           
                            
        def transicionEstados(self, dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)
                if(me_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado <= dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = dia + random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2    #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado =dia+ random.randint(int(mediaduracion-desvduracion),int(mediaduracion+desvduracion)) #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 
                                     
                

In [19]:

class Simulador:
        def __init__(self, numpersonas,personasedificio):           
            self.ciudad=crearciudadv0(numpersonas,personasedificio)
            self.dia=0 #se inicializa el dia con un valor de 0 
            self.hora=0 #se inicializa las horas con un valor de 0 
            self.numpersonasinicial=numpersonas 
            self.numpersonas=numpersonas
            self.numedificios=len(self.ciudad)
            self.capacidades=np.zeros(self.numedificios)
            self.CatalogoPersonas=[]
            self.cementerio=[]
            for i in range(self.numedificios-1,-1,-1):
                self.capacidades[i]=int(len(self.ciudad[i])) #establecemos las capacidades de los edificios
                for j in range (len(self.ciudad[i])):
                    self.CatalogoPersonas.append(self.ciudad[i][j])
                    
            self.capacidades=self.capacidades.astype(int)
            print(self.capacidades)
            #for i in range(len(self.CatalogoPersonas)):
            #    print(self.CatalogoPersonas[i])
            
                
           
        def __str__(self):
            #Devuelve un string con la informacion util del simulador
            return str(self.dia) + "," + str(self.hora) + "," + str(self.numpersonas) + "," +str(self.numedificios)  
        
        def actualizar_catalogo(self):#actualizamos los datos de las personas ordenadas por id
            nuevoCatalogo=[]
            for i in range(self.numedificios):
                for j in range (len(self.ciudad[i])):
                    nuevoCatalogo.append(self.ciudad[i][j]) #primero cogemos todas las personas sin ordenar
                    self.CatalogoPersonas = sorted(nuevoCatalogo, key=lambda x: x.idpersona, reverse=False)
            
        #Hace que el simulador avance una hora , con sus consecuencias en las personas
        def pasar_tiempo(self,mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            if ((self.hora % 8)==0): #barajar a las personas cada 8 horas( hacen shuffle)
                self.moverpersonas2()
                #print("owo")
            for j in range(len(self.ciudad)): #recorre los edificios 
                for i in range(len(self.ciudad[j])): #recorre las personas 
                    self.ciudad[j][i].infectar(0.4,self.ciudad,mediaincubacion,desvincubacion,self.dia) #pasa a infectar 
            if (self.hora<23):
                self.hora+=1
            else:#comienza un nuevo dia
                self.hora=0
                self.dia+=1
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):                     
                        self.ciudad[j][i].transicionEstados(self.dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion)
                        
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])-1,-1,-1):   
                        if (self.ciudad[j][i].estado=="muerto"):
                            
                            self.cementerio.append(self.ciudad[j].pop(i))
                            self.numpersonas-=1
                self.cementerio=sorted(self.cementerio, key=lambda x: x.idpersona, reverse=False)
                            
        
        #Función que devuelve el total de personas sanas o que ya pasaron la enfermedad de la ciudad
        def personas_sanas(self):
            contador_sanos=0
            for j in range(len(self.ciudad)): #se recorre los edificios de la ciudad
                    for i in range(len(self.ciudad[j])): #se va recorriendo las personas de los edificios 
                        if self.ciudad[j][i].esta_sano():
                            contador_sanos+=1
            return contador_sanos
        
        def cogerpersona(self, idpersona):
            self.actualizar_catalogo()
            return self.CatalogoPersonas[idpersona]
        
        
        
        def moverpersonas(self):#Baraja a todas las personas de la ciudad indiscriminadamente
                                #Vamos a crear una nueva ciudad utilizando las personas actuales
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosSinVaciar=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            for i in range(self.numedificios-1,-1,-1):
                if (len(self.ciudad[i])==0):
                    del self.ciudad[i]
                    edificiosSinVaciar-=1
            
            for i in range(self.numedificios):
                edificionuevo = [] #El edificio actual de la ciudad nueva.
                
                for j in range(self.capacidades[i]):
                    if personasColocadas==self.numpersonas:
                        break
                    
                    if edificiosSinVaciar>1:
                        edificioescogido=random.randint(0,edificiosSinVaciar)
                    else:
                        edificioescogido=0
                    edificionuevo.append(self.ciudad[edificioescogido].pop(0))
                    edificionuevo[j].lugar=i
                    personasColocadas+=1
                    if (personasColocadas==self.numpersonas):
                        break
                    if (len(self.ciudad[edificioescogido])==0):
                        del  self.ciudad[edificioescogido]
                        edificiosSinVaciar-=1
            
                ciudadnueva.append(edificionuevo)
            self.ciudad=ciudadnueva
            
        def moverpersonas2(self):
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosAntiguos=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            opciones=[]
            for i in range(self.numedificios):
                edificionuevo=[]
                ciudadnueva.append(edificionuevo)
                opciones.append(i)
            
            personasColocadas=0
            edificiosLlenos=self.numedificios
            
            for i in range(self.numedificios):
                for j in range(len(self.ciudad[i])):
                    if (len(self.ciudad[i])==0):
                        break
                    edificioALlenar=random.choice(opciones)
                    ciudadnueva[edificioALlenar].append(self.ciudad[i].pop(0))
                    if (len(ciudadnueva[edificioALlenar])==self.capacidades[edificioALlenar]):
                        opciones.remove(edificioALlenar)
            self.ciudad=ciudadnueva
            
            
        #Función que contagia a la primera persona (la deja sintomática), se usa para crear pacientes 0
        def contagio_fijo(self):
            self.ciudad[0][0].estado=self.ciudad[0][0].estadosposibles[3]
            self.ciudad[0][0].cambioEstado=15
            
            
        def mostrarcementerio(self):#pues eso
            print(f"Total de muertos:{len(self.cementerio)}")
            for j in range(len(self.cementerio)):
                print(self.cementerio[j])

In [20]:
#Funcion que se encarga de generar la edad de manera aleatoria
def generador_edad():
        n = random.randint(0,110) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [21]:
#Funcion que se encarga de generar numeros aleatorios 
def generador_numeros(longitud):
    numero=0
    for i in range(0,longitud): 
        n = random.randint(0,9)
        numero=10*numero+n
    return numero


In [22]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni():
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni
    input=generador_numeros(8)
    #Se consigue que toda la secuencia de numeros sea 1 solo
    
    #Se calcula la letra que ha de ser adicionada
    valor =int(input / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = input - valor;
    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia 
    DNI=str(input)+letras[valor]
    
    return DNI

In [23]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        #print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            #print(f"contador del edificio:{contadoredificio}")
            #print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            column.append(Persona(capacidadusada,generador_edad(),edificioactual, creaDni()))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
    
    for j in range(len(ciudad[i])):
        print(ciudad[i][j])
    

In [30]:
MiSimulador=Simulador(800,20) #numPersonas, personasEdificio?? 
print(MiSimulador)
MiSimulador.contagio_fijo() #se crea un paciente que resulta ser el 0 
for i in range(1200):
    '''
    Datos inventados:
    MediaIncubación:7 dias
    DesvIncubación:3 dias
    MediaDuracion:14 dias
    DescDuracion:5 dias
    '''
    MiSimulador.pasar_tiempo(7,3,14,5)
    if MiSimulador.hora==0:
        print("--------")
        print(" ")
        print(" ")
        print(MiSimulador.dia)
        print(MiSimulador.capacidades)
        print(MiSimulador)
        printearciudad(MiSimulador.ciudad)
        print(MiSimulador.personas_sanas())
        print(" ")
        print(" ")
        print("--------")
        print(" ")
        print(" ")
        MiSimulador.mostrarcementerio()
        print(" ")
        print(" ")
        print("--------")

[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
0,0,800,24
Total de vivos:800
owo
Total de vivos:800
owo
Total de vivos:800
owo
--------
 
 
1
[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
1,0,800,24
101,sano,2,20406626Z,5
459,sano,13,32400812E,31
585,sano,17,81581376P,78
483,sano,13,83016117D,109
222,sano,7,50062815A,45
391,sano,11,3735783P,97
184,sano,6,82622408Z,58
147,sano,4,82843460J,88
180,sano,6,41620381H,74
342,sano,10,3443054T,63
541,sano,15,83450475N,65
290,sano,9,57078345G,23
654,sano,19,54457340X,20
365,sano,10,53434813V,29
24,sano,0,41068606J,0
339,sano,10,70258511C,10
359,sano,10,30226400F,90
724,incubando,21,10507634S,81
488,sano,14,77516374A,72
580,sano,16,10656364G,97
6,sano,0,27440504D,15
370,sano,11,64668158R,6
458,sano,13,28746844H,71
480,sano,13,78425154F,9
514,incubando,14,44470811N,106
170,sano,6,67171028B,14
165,incubando,5,40704388T,15
142,sano,4,84824315Q,31
722,incubando,21,8003883K,65
777,sano,22,68685572

697,sano,19,84343042H,41
103,sano,3,53682650Y,66
314,sano,9,60713438Q,16
47,sano,0,80143404L,75
507,sano,14,38638180C,101
531,incubando,14,51814141V,73
0,sintomatico,0,58545756S,82
242,sano,7,61520668S,2
708,sano,21,41041000F,62
588,sano,17,33037567E,105
660,sano,19,53582754E,14
474,sano,13,32351237N,76
478,sano,13,32315081N,49
730,sano,21,64407178W,84
643,sano,19,52075600N,16
510,sano,14,73745811Z,37
747,sano,22,42503043D,11
569,sano,16,78207272G,17
595,sano,17,78036436N,44
446,sano,13,25507474Z,16
523,sano,14,63155134D,28
291,sano,9,36347080L,76
307,sano,9,15186457V,19
381,sano,11,80834866D,64
680,sano,19,10017088J,15
670,sano,19,4774161M,43
210,sano,6,58157530Y,50
144,sano,4,4434505J,66
498,sano,14,44067287T,17
10,sano,0,24303755T,47
349,sano,10,55483022F,27
429,sano,12,50741157Y,33
304,sano,9,72678453V,72
208,sano,6,54782604P,22
78,sano,2,74521354C,86
774,sano,22,13344285F,90
108,sano,3,11525504C,35
107,sano,3,16037162K,9
548,sano,16,5380603Y,38
35,sano,0,22864266N,58
140,sano,4,15

Total de vivos:800
owo
Total de vivos:800
owo
--------
 
 
4
[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
4,0,800,24
110,sano,3,56207060M,32
1,sano,0,83533031K,28
183,sano,6,23803381Z,27
496,incubando,14,68234184S,14
470,incubando,13,11472214K,15
794,sano,23,6317134T,0
734,incubandoContagioso,21,48734525D,90
634,sano,18,61043255J,2
374,incubando,11,75038067S,29
237,incubando,7,131356A,26
166,sano,5,18364083D,66
196,sano,6,36415815F,14
193,sano,6,54781063P,36
364,incubando,10,24546206P,14
291,incubando,9,36347080L,76
385,sano,11,37164244Q,55
332,sano,9,55477008L,19
773,sano,22,17547421P,4
188,sano,6,53115512W,29
622,incubando,18,25012772L,41
156,incubando,4,3605312Q,100
772,incubando,22,64725708M,23
377,incubandoContagioso,11,30634564J,91
629,sano,18,35673580M,19
131,incubando,3,66762110X,41
210,sano,6,58157530Y,50
273,sano,8,6176335F,50
3,sano,0,22514687X,7
716,sano,21,85082813V,58
198,sano,6,56507212F,4
701,incubando,20,7841625M,58
758,sano,22,43801577D,43

5
[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
5,0,799,24
312,incubando,9,58803767N,57
276,incubandoContagioso,8,12874426S,49
225,incubandoContagioso,7,22343242F,12
297,sano,9,74055887G,109
148,incubando,4,40747102A,57
626,incubando,18,88466142T,31
550,incubando,16,46215877E,84
494,sano,14,55122660D,30
570,sano,16,32068706J,15
13,sano,0,83812365C,103
206,sano,6,34141437F,8
741,incubandoContagioso,22,34861370V,71
308,sano,9,27033523J,42
185,sano,6,63245045J,97
4,sano,0,77386236E,39
663,incubando,19,80606884A,15
164,incubando,5,87820643L,16
489,incubando,14,65786700F,94
275,sano,8,64456003K,85
608,sano,17,13034475F,43
478,sano,13,32315081N,49
573,sano,16,42831502M,89
767,incubando,22,46144570S,33
534,sano,14,27116521G,97
678,incubando,19,16543186E,108
190,incubando,6,7676574W,79
646,incubandoContagioso,19,54722801M,94
180,incubando,6,41620381H,74
54,incubandoContagioso,0,23518541Y,87
133,incubando,3,44506764Q,3
796,incubando,23,45037505D,107
492,sano,14,63260

548,incubando,16,5380603Y,38
21,incubando,0,6521841F,94
552,incubandoContagioso,16,43685324K,0
9,incubandoContagioso,0,8715678N,52
387,incubando,11,26452724X,108
30,incubando,0,27104800J,50
547,incubando,16,58884851K,30
622,incubando,18,25012772L,41
627,incubando,18,55234238Z,49
273,sano,8,6176335F,50
590,incubandoContagioso,17,38651107K,32
553,incubandoContagioso,16,14316265F,39
543,incubando,16,32420042R,85
583,incubando,16,84723023Q,93
20,sano,0,37884675H,109
788,incubando,23,22642841P,59

211,incubando,6,88712278J,40
607,incubando,17,86614236P,41
49,incubando,0,47468870K,69
61,incubando,1,13711313R,56
709,incubandoContagioso,21,34624670X,18
424,incubando,12,67372210N,75
16,incubandoContagioso,0,16782363E,85
699,incubando,19,11855085B,37
331,incubandoContagioso,9,65630736Y,81
475,incubandoContagioso,13,53311573B,57
685,incubando,19,45582014Q,11
732,incubando,21,18407685A,44
243,incubando,7,72650024Q,66
145,incubando,4,81762376K,47
370,incubando,11,64668158R,6
74,incubando,1,50807808

...

owo
--------
 
 
43
[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
43,0,659,24
591,inmune,17,68187800E,29
289,inmune,9,44867222H,100
418,inmune,12,50654334P,76
339,inmune,10,70258511C,10
653,inmune,19,25838786B,13
70,inmune,1,52115567M,106
373,inmune,11,81606018V,39
114,inmune,3,25248141Y,79
505,inmune,14,84528663Y,88
493,inmune,14,55067784B,38
689,inmune,19,28224285L,23
566,inmune,16,28067673Z,54
112,inmune,3,52178083F,78
368,inmune,11,50155574A,57
794,inmune,23,6317134T,0
141,inmune,4,5783323L,25
408,inmune,12,64044002L,56
296,inmune,9,71664010N,2
121,inmune,3,22427166G,15
533,inmune,14,73153885Q,80
95,inmune,2,8670837K,107
349,inmune,10,55483022F,27
717,inmune,21,31422400F,102
492,inmune,14,63260370C,96
510,inmune,14,73745811Z,37

555,inmune,16,6830406G,9
41,inmune,0,8125650A,4
47,inmune,0,80143404L,75
568,inmune,16,56440064L,86
629,inmune,18,35673580M,19
465,inmune,13,37278302V,10
792,inmune,23,56637736Y,93
782,inmune,23,26213535K,109
240,inmune,7,7756456

532,muerto,14,57711664H,81
539,muerto,15,15252408G,78
542,muerto,15,82448806Q,73
573,muerto,16,42831502M,89
574,muerto,16,48714634J,55
575,muerto,16,18301735Z,73
583,muerto,16,84723023Q,93
584,muerto,16,13518416M,80
589,muerto,17,80888835C,16
594,muerto,17,33343276S,106
596,muerto,17,28807662R,79
602,muerto,17,68184074E,81
605,muerto,17,63578660J,59
606,muerto,17,5431208B,68
613,muerto,17,30875214Z,99
614,muerto,17,7363040G,97
625,muerto,18,17648114F,76
633,muerto,18,35562306M,75
637,muerto,18,67272520G,87
646,muerto,19,54722801M,94
657,muerto,19,71435328L,96
658,muerto,19,56634273Q,76
666,muerto,19,88164542E,13
667,muerto,19,20070457J,87
668,muerto,19,73631517F,83
669,muerto,19,18368023Q,80
671,muerto,19,82576857A,86
676,muerto,19,33633088G,86
688,muerto,19,67775126Z,72
693,muerto,19,43222806X,62
713,muerto,21,44645111H,91
714,muerto,21,70410637R,70
721,muerto,21,28232632V,69
724,muerto,21,10507634S,81
728,muerto,21,55718082F,55
729,muerto,21,11470867P,79
735,muerto,21,17078315X,99
73

--------
 
 
47
[58 17 27 34 23  9 47 35 28 57 33 32 37 48 54  4 42 30 28 58  6 30 45 18]
47,0,659,24
411,inmune,12,33641081Q,23
473,inmune,13,17087442Y,79
710,inmune,21,81208810L,41
282,inmune,9,60144213H,65
10,inmune,0,24303755T,47
703,inmune,20,41337107N,59
297,inmune,9,74055887G,109
530,inmune,14,43080372S,109
401,inmune,12,83027607E,39
171,inmune,6,1561851J,48
638,inmune,18,58176636E,11
429,inmune,12,50741157Y,33
651,inmune,19,43253332S,80
186,inmune,6,22324507V,12
229,inmune,7,23200061F,47
544,inmune,16,44803833V,54
423,inmune,12,11803661S,16
358,inmune,10,60232404G,88
635,inmune,18,77614818F,25
753,inmune,22,81833877S,100
33,inmune,0,50642324G,84
280,inmune,9,75486542J,51
300,inmune,9,33466573D,65
74,inmune,1,50807808A,65
173,inmune,6,16628614M,106
382,inmune,11,40520646M,5
323,inmune,9,5853885V,56
743,inmune,22,13137015J,77
760,inmune,22,77663673X,27
215,inmune,7,48218127F,47
700,inmune,19,20123850T,105
686,inmune,19,68724075Y,75
785,inmune,23,28736244K,99
761,inmune,22,5757850

Total de muertos:141
0,muerto,0,58545756S,82
13,muerto,0,83812365C,103
20,muerto,0,37884675H,109
21,muerto,0,6521841F,94
22,muerto,0,3305557C,32
26,muerto,0,38657256Y,95
28,muerto,0,16716483Z,36
32,muerto,0,36414000D,109
37,muerto,0,12550662E,104
40,muerto,0,17476585N,15
43,muerto,0,35571503W,85
44,muerto,0,44081171S,107
49,muerto,0,47468870K,69
52,muerto,0,40177171N,104
53,muerto,0,17727702S,14
59,muerto,1,84665713E,95
60,muerto,1,26536623M,77
63,muerto,1,6688045J,80
67,muerto,1,68137068M,58
84,muerto,2,43726481P,61
90,muerto,2,27825347Q,82
93,muerto,2,54075878A,92
124,muerto,3,85213641K,90
128,muerto,3,86547683V,106
130,muerto,3,23425123Z,101
134,muerto,3,42133006L,5
142,muerto,4,84824315Q,31
144,muerto,4,4434505J,66
150,muerto,4,34584851G,76
152,muerto,4,75228387X,65
156,muerto,4,3605312Q,100
157,muerto,4,48667355E,67
166,muerto,5,18364083D,66
170,muerto,6,67171028B,14
181,muerto,6,63867531G,107
182,muerto,6,83020180R,96
183,muerto,6,23803381Z,27
185,muerto,6,63245045J,97
201,muerto

697,inmune,19,84343042H,41
521,inmune,14,64073261E,15
341,inmune,10,85328318C,46
437,inmune,13,46281081K,22
491,inmune,14,14570350B,67
197,inmune,6,60808621W,48
690,inmune,19,36034415Q,36
103,inmune,3,53682650Y,66

162,inmune,5,75873282P,46
222,inmune,7,50062815A,45
500,inmune,14,6775235X,77
498,inmune,14,44067287T,17
314,inmune,9,60713438Q,16
422,inmune,12,5358846F,40
743,inmune,22,13137015J,77
106,inmune,3,45234360F,41
173,inmune,6,16628614M,106
487,inmune,14,76487016X,70
659,inmune,19,31374642C,95
644,inmune,19,57762557N,55
581,inmune,16,75458117Q,49
203,inmune,6,33265172L,20
159,inmune,5,73127223B,27
217,inmune,7,70102640C,97
611,inmune,17,83778560W,73
11,inmune,0,62203511B,70
746,inmune,22,4132127Q,34
114,inmune,3,25248141Y,79
122,inmune,3,12250250J,23
247,inmune,7,65536205M,105
640,inmune,18,31882368K,67
474,inmune,13,32351237N,76
148,inmune,4,40747102A,57
592,inmune,17,43082577N,64
754,inmune,22,82114488A,64
243,inmune,7,72650024Q,66
435,inmune,12,40643573C,53
791,inmune,23,1221

In [25]:
printearEdificio(MiSimulador.ciudad,0)

68,inmune,3,56711266M,22


In [26]:
printearciudad(MiSimulador.ciudad)

68,inmune,3,56711266M,22

22,inmune,1,44622371W,57
18,inmune,1,81356210N,8
76,inmune,4,88524085Y,59
23,inmune,1,86670864X,4
32,inmune,1,42454783A,11
20,inmune,1,53428358W,78
3,inmune,1,27102032M,34
83,inmune,4,77414261X,56
72,inmune,4,25210143G,20
30,inmune,1,58846701M,39
48,inmune,2,1162457Z,63
88,inmune,4,33846622Y,15
57,inmune,2,71436206T,22
11,inmune,1,48667561K,107
93,inmune,4,68352384H,80
4,inmune,1,11776370W,28
43,inmune,2,4277068B,35
98,inmune,5,46154545P,101
97,inmune,5,60588057P,32
19,inmune,1,15120414F,10
65,inmune,3,34372778Z,100

8,inmune,1,53054467E,46
36,inmune,2,26306750V,18
89,inmune,4,34758580Z,78
80,inmune,4,72888650V,86
67,inmune,3,86035467B,91
53,inmune,2,8833281Q,63
15,inmune,1,2846257F,85
50,inmune,2,13316802D,39
84,inmune,4,60008807J,36
2,inmune,1,43602763F,64
29,inmune,1,80645358K,47
55,inmune,2,41147331D,35
64,inmune,3,58076113D,101
44,inmune,2,3608884T,92
35,inmune,1,41825024F,92
59,inmune,2,30124532Y,9
78,inmune,4,18158546T,9
25,inmune,1,5302348C,34
79,inmun

In [27]:
MiSimulador.actualizar_catalogo()
for i in range(len(MiSimulador.CatalogoPersonas)):
                print(MiSimulador.CatalogoPersonas[i])
            

0,inmune,0,11243867H,25
1,inmune,1,56224405P,70
2,inmune,1,43602763F,64
3,inmune,1,27102032M,34
4,inmune,1,11776370W,28
5,inmune,1,87008361M,2
7,inmune,1,55601061X,7
8,inmune,1,53054467E,46
10,inmune,1,86644448K,22
11,inmune,1,48667561K,107
12,inmune,1,2470853D,48
13,inmune,1,24375770W,21
14,inmune,1,75147732Q,71
15,inmune,1,2846257F,85
18,inmune,1,81356210N,8
19,inmune,1,15120414F,10
20,inmune,1,53428358W,78
22,inmune,1,44622371W,57
23,inmune,1,86670864X,4
25,inmune,1,5302348C,34
28,inmune,1,82581208F,67
29,inmune,1,80645358K,47
30,inmune,1,58846701M,39
32,inmune,1,42454783A,11
33,inmune,1,76654370Q,35
34,inmune,1,63281802Q,48
35,inmune,1,41825024F,92
36,inmune,2,26306750V,18
37,inmune,2,50446516H,104
39,inmune,2,43800534R,6
41,inmune,2,40557607M,49
42,inmune,2,76482600X,35
43,inmune,2,4277068B,35
44,inmune,2,3608884T,92
45,inmune,2,47076434B,42
47,inmune,2,61775203D,34
48,inmune,2,1162457Z,63
49,inmune,2,11447256H,44
50,inmune,2,13316802D,39
51,inmune,2,16025612V,56
53,inmune,2,88332

In [28]:
personitaputa=MiSimulador.cogerpersona(0)
print(personitaputa.contadorInfecciones)

4


In [167]:
MiSimulador=Simulador(100,10) 
print(MiSimulador.capacidades)
print(len(MiSimulador.ciudad))

[23  1 29  7 22 16  2]
0,sano,0,87204200E,7
1,sano,0,81351675P,65
2,sano,0,7541462S,84
3,sano,0,2734837E,49
4,sano,0,67118311X,62
5,sano,0,63653820D,41
6,sano,0,56840305Z,104
7,sano,0,71438685H,88
8,sano,0,37235031D,2
9,sano,0,24211256F,28
10,sano,0,76816004Y,83
11,sano,0,25172814G,49
12,sano,0,75582883F,33
13,sano,0,65767330A,93
14,sano,0,22875655Q,88
15,sano,0,11256111A,85
16,sano,0,87500174D,37
17,sano,0,23444251Y,63
18,sano,0,50162442V,58
19,sano,0,52664477C,99
20,sano,0,10176827V,22
21,sano,0,57341107Z,72
22,sano,0,32625576F,67
23,sano,1,76036012J,70
24,sano,2,26170230W,84
25,sano,2,30847235A,11
26,sano,2,78242507A,47
27,sano,2,12318218Q,13
28,sano,2,3440412A,53
29,sano,2,32441324P,31
30,sano,2,26126301A,94
31,sano,2,28610526K,51
32,sano,2,27767452N,105
33,sano,2,50417272F,78
34,sano,2,2504088D,32
35,sano,2,57177760J,40
36,sano,2,31066118H,88
37,sano,2,76265586R,79
38,sano,2,68173116N,58
39,sano,2,35280872E,52
40,sano,2,34084064L,36
41,sano,2,6287818D,23
42,sano,2,11550463R,81
43,